**Disclaimer:** The distributed notebook only serves for demonstration purposes. Jupyter notebooks are not recommended for preCICE development or deployment. Feel free to convert this notebook to a plain python script for improved error messages using `jupyter nbconvert --to python Solver*.ipynb`

In [ ]:
def dummySolidSolver(x, y, dt):
    return x + dt * y**2

In [ ]:
import precice
precice_config = "precice-config-both-init.xml"
#precice_config = "precice-config-init-serial.xml"
#precice_config = "precice-config-experimental-both-init.xml"
interface = precice.Interface("Solid", precice_config, 0,1)
meshId = interface.get_mesh_id("Solid-Mesh")
vertexId = interface.set_mesh_vertex(meshId, [1,2,3])
readDataId = interface.get_data_id("Force", meshId)
writeDataId = interface.get_data_id("Displacement", meshId)

In [ ]:
precice_dt = interface.initialize()

### Initialize data
optional, if `initialize = "false"` or not given

In [ ]:
initDisplacement = 2
writeData = initDisplacement
interface.write_scalar_data(writeDataId, vertexId, writeData)

In [ ]:
interface.mark_action_fulfilled(precice.action_write_initial_data())

In [ ]:
interface.initialize_data()

### First window, first iteration

require to write checkpoint, when entering window:

In [ ]:
interface.mark_action_fulfilled(precice.action_write_iteration_checkpoint())

In [ ]:
readData = interface.read_scalar_data(readDataId, vertexId)
print("readData = {}".format(readData))

In [ ]:
writeData = dummySolidSolver(writeData, readData, precice_dt)

In [ ]:
print("writeData = {}".format(writeData))
interface.write_scalar_data(writeDataId, vertexId, writeData)

In [ ]:
precice_dt = interface.advance(precice_dt)
print("precice_dt = {}".format(precice_dt))

In [ ]:
interface.is_time_window_complete()

### First window, second iteration
require to read checkpoint, when repeating window:

In [ ]:
interface.mark_action_fulfilled(precice.action_read_iteration_checkpoint())

In [ ]:
readData = interface.read_scalar_data(readDataId, vertexId)
print("readData = {}".format(readData))

In [ ]:
writeData = dummySolidSolver(writeData, readData, precice_dt)

In [ ]:
print("writeData = {}".format(writeData))
interface.write_scalar_data(writeDataId, vertexId, writeData)

In [ ]:
precice_dt = interface.advance(precice_dt)
print("precice_dt = {}".format(precice_dt))

In [ ]:
interface.is_time_window_complete()

### Second window:
... same procedure as before.

In [ ]:
window = 2
iteration = 1

while interface.is_coupling_ongoing():
    if interface.is_action_required(precice.action_write_iteration_checkpoint()):
        interface.mark_action_fulfilled(precice.action_write_iteration_checkpoint())
        
    readData = interface.read_scalar_data(readDataId, vertexId)
    print("readData = {}".format(readData))
    
    writeData = dummySolidSolver(writeData, readData, precice_dt)
    
    print("writeData = {}".format(writeData))
    interface.write_scalar_data(writeDataId, vertexId, writeData)
    
    precice_dt = interface.advance(precice_dt)
    print("precice_dt = {}".format(precice_dt))
    
    if interface.is_time_window_complete():
        window += 1
        iteration = 1
    if interface.is_action_required(precice.action_read_iteration_checkpoint()):
        iteration += 1
    
    if interface.is_coupling_ongoing():
        print("window={}".format(window))
        print("iteration={}".format(iteration))
        
    if interface.is_action_required(precice.action_read_iteration_checkpoint()):
        interface.mark_action_fulfilled(precice.action_read_iteration_checkpoint())
        
print("Done!")
interface.finalize()